# 01_EDA – Exploratory Data Analysis on Login Event Data

This notebook performs exploratory data analysis (EDA) on the RBA login dataset to understand user login behaviors, identify key patterns, and prepare the data for anomaly detection using machine learning.

### Objectives:
- Inspect dataset structure and types
- Analyze login distributions by country, time, and user behavior
- Visualize geo-based and temporal login trends
- Identify potential indicators of malicious activity (e.g., rare locations, odd login hours)
- Guide feature engineering for downstream ML models